In [1]:
import pandas as pd
import numpy as np
import gc
import h2o
from h2o.automl import H2OAutoML
from autogluon.tabular import TabularPredictor
from sklearn.preprocessing import LabelEncoder

# 데이터 전처리

In [ ]:
# 데이터 분할(폴더) 구분
data_splits = ["train", "test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
    "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
    "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
    "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
    "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
    "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
    "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"open/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

In [3]:
# 데이터 유형별 설정 
info_categories = ["customer", "credit", "sales", "billing", "balance", "channel", "marketing", "performance"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

In [ ]:
#### Train ####

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")


customer_train_df = train_dfs["customer_train_df"]
credit_train_df   = train_dfs["credit_train_df"]
sales_train_df    = train_dfs["sales_train_df"]
billing_train_df  = train_dfs["billing_train_df"]
balance_train_df  = train_dfs["balance_train_df"]
channel_train_df  = train_dfs["channel_train_df"]
marketing_train_df= train_dfs["marketing_train_df"]
performance_train_df = train_dfs["performance_train_df"]

gc.collect()

In [ ]:
#### Test ####

# test 데이터에 대해 train과 동일한 방법 적용
test_dfs = {}

for prefix in info_categories:
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")


customer_test_df = test_dfs["customer_test_df"]
credit_test_df   = test_dfs["credit_test_df"]
sales_test_df    = test_dfs["sales_test_df"]
billing_test_df  = test_dfs["billing_test_df"]
balance_test_df  = test_dfs["balance_test_df"]
channel_test_df  = test_dfs["channel_test_df"]
marketing_test_df= test_dfs["marketing_test_df"]
performance_test_df = test_dfs["performance_test_df"]

gc.collect()

In [ ]:
#### Train ####

train_df = customer_train_df.merge(credit_train_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: train_step1, shape:", train_df.shape)
del customer_train_df, credit_train_df
gc.collect()

# 이후 merge할 데이터프레임 이름과 단계 정보를 리스트에 저장
merge_list = [
    ("sales_train_df",    "Step2"),
    ("billing_train_df",  "Step3"),
    ("balance_train_df",  "Step4"),
    ("channel_train_df",  "Step5"),
    ("marketing_train_df","Step6"),
    ("performance_train_df", "최종")
]

# 나머지 단계 merge
for df_name, step in merge_list:
    # globals()로 동적 변수 접근하여 merge 수행
    train_df = train_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"{step} 저장 완료: train_{step}, shape:", train_df.shape)
    # 사용한 변수는 메모리 해제를 위해 삭제
    del globals()[df_name]
    gc.collect()

In [ ]:
#### Test ####

test_df = customer_test_df.merge(credit_test_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: test_step1, shape:", test_df.shape)
del customer_test_df, credit_test_df
gc.collect()

# 이후 merge할 데이터프레임 이름과 단계 정보를 리스트에 저장
merge_list = [
    ("sales_test_df",    "Step2"),
    ("billing_test_df",  "Step3"),
    ("balance_test_df",  "Step4"),
    ("channel_test_df",  "Step5"),
    ("marketing_test_df","Step6"),
    ("performance_test_df", "최종")
]

# 나머지 단계 merge
for df_name, step in merge_list:
    # globals()로 동적 변수 접근하여 merge 수행
    test_df = test_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"{step} 저장 완료: test_{step}, shape:", test_df.shape)
    # 사용한 변수는 메모리 해제를 위해 삭제
    del globals()[df_name]
    gc.collect()

In [ ]:
feature_cols = [col for col in train_df.columns if col not in ["ID", "Segment"]]

X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()

# 타깃 라벨 인코딩
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

In [ ]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

X_test = test_df.copy()

encoders = {}  # 각 컬럼별 encoder 저장

for col in categorical_features:
    le_train = LabelEncoder()
    X[col] = le_train.fit_transform(X[col])
    encoders[col] = le_train
    unseen_labels_val = set(X_test[col]) - set(le_train.classes_)
    if unseen_labels_val:
        le_train.classes_ = np.append(le_train.classes_, list(unseen_labels_val))
    X_test[col] = le_train.transform(X_test[col])

In [ ]:
gc.collect()

# 데이터 학습

In [2]:
data_path = "D:/workspace/data/dacon/"

In [3]:
train_df = pd.read_csv(data_path + "merge_train.csv")

In [4]:
target = "target"  # 대회 타겟 컬럼
features = [col for col in train_df.columns if col != target]

## H2O 서버 실행

In [6]:
# 🚀 H2O 서버 시작
h2o.init(max_mem_size="16G", nthreads=-1)

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,20 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,4 months and 18 days
H2O_cluster_name:,H2O_from_python_JINWOO_yg7oyq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,15.51 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


## 전체 데이터 실행 (메모리 부족)

In [ ]:
# ✅ H2O AutoML 실행 (F1 Score 최적화)
df_h2o = h2o.H2OFrame(train_df)

aml = H2OAutoML(
    max_runtime_secs=7200,  # 2시간 제한
    max_models=20,
    stopping_metric="F1",
    balance_classes=True,
    seed=42
)
aml.train(x=features, y=target, training_frame=df_h2o)

## 샘플링 데이터 실행

In [11]:

# ✅ RAM 최적화: 샘플링 적용 (50% 데이터만 사용)
train_df_sampled = train_df.sample(frac=0.5, random_state=42)


In [12]:
# 결측치가 있는 컬럼 확인
missing_cols = train_df_sampled.columns[train_df_sampled.isna().sum() > 0]

# 결측치가 있는 컬럼 출력
print("결측치가 있는 컬럼 목록:")
print(missing_cols)

결측치가 있는 컬럼 목록:
Index(['최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', 'RV신청일자',
       '최종카드론_금융상환방식코드', '최종카드론_대출일자', '연체일자_B0M', '혜택수혜율_R3M', '혜택수혜율_B0M'],
      dtype='object')


In [13]:
train_df_cleaned = train_df_sampled.drop(columns=missing_cols)

In [ ]:

# ✅ RAM 최적화: H2OFrame 변환 시 chunk_size 설정
df_h2o = h2o.H2OFrame(train_df_cleaned)  

In [ ]:


# ✅ Feature Selection: 중요 컬럼만 학습
features = df_h2o.columns[:500]  # 메모리 초과 방지

# ✅ H2O AutoML 실행
aml = H2OAutoML(max_runtime_secs=3600, stopping_metric="mean_per_class_error", balance_classes=True)
aml.train(x=features, y=target, training_frame=df_h2o)

# ✅ 결과 확인
print(aml.leaderboard)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |█
15:16:35.512: AutoML: XGBoost is not available; skipping it.
15:16:35.607: _train param, Dropping bad and constant columns: [시장연체상환여부_R3M, 증감_RP건수_건강_전월, 이용건수_당사기타_B0M, 이용개월수_당사페이_R6M, 이용금액_당사기타_R6M, 상품관련면제카드수_B0M, 할인금액_기본연회비_B0M, RP건수_건강_B0M, RP건수_유선방송_B0M, 이용건수_당사페이_R3M, 이용금액_당사기타_B0M, 할부건수_부분_3M_R12M, 연회비할인카드수_B0M, 이용건수_당사페이_R6M, 이용금액_당사페이_R3M, 할인금액_제휴연회비_B0M, 기타면제카드수_B0M, 이용금액_당사기타_R3M, RP후경과월_건강, 이용건수_당사기타_R6M, 이용건수_당사페이_B0M, 우수회원면제카드수_B0M, 임직원면제카드수_B0M, 할부건수_부분_14M_R12M, 할부금액_부분_3M_R12M, 시장연체상환여부_R6M, 할부건수_부분_6M_R12M, 납부_렌탈료이용금액, 이용금액_부분무이자_B0M, 납부_유선방송이용금액, 이용건수_부분무이자_B0M, 할부금액_부분_6M_R12M, 이용카드수_체크_가족, 여유_여행이용금액, 증감_RP건수_유선방송_전월, 이용금액_당사페이_R6M, 이용금액_R3M_체크_가족, 납부_건강연금이용금액, RP후경과월_유선방송, 이용건수_당사기타_R3M, 이용금액_당사페이_B0M]

████
15:17:44.738: _train param, Dropping bad and constant columns: [시장연체상환여부_R3M, 증감_RP건수_건강_전월, 이용건수_당사기타_B0M, 이용개월수_당사페이_R6M, 이용금액_당사기타_R6M, 상품관련면제카드

OSError: Job with key $03017f00000132d4ffffffff$_966ea47535242a919e83cd2ef2a14eb5 failed with an exception: water.exceptions.H2OAutoMLException: Aborting AutoML after too many consecutive model failures
stacktrace: 
water.exceptions.H2OAutoMLException: Aborting AutoML after too many consecutive model failures
	at ai.h2o.automl.AutoML.learn(AutoML.java:776)
	at ai.h2o.automl.AutoML.run(AutoML.java:494)
	at ai.h2o.automl.H2OJob$1.compute2(H2OJob.java:33)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1704)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:976)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1479)
	at jsr166y.ForkJoinWorkerThread.run(ForkJoinWorkerThread.java:104)
